In [91]:
import xarray as xr

import numpy as np
import scipy

import os
import timeit
import time


In [92]:
models = [
        # 'IPSL-CM5A-MR', # 1
        'GFDL-CM3',      # 2
        # 'GISS-E2-H',    # 3
        # 'bcc-csm1-1',   # 4
        # 'CNRM-CM5',     # 5
        # 'CCSM4',        # 6
        # 'HadGEM2-AO',   # 7
        # 'BNU-ESM',      # 8
        # 'EC-EARTH',     # 9
        # 'FGOALS-g2',    # 10
        # 'MPI-ESM-MR',   # 11
        # 'CMCC-CM',      # 12
        # 'inmcm4',       # 13
        # 'NorESM1-M',    # 14
        # 'CanESM2',      # 15 
        # 'MIROC5',       # 16
        # 'HadGEM2-CC',   # 17
        # 'MRI-CGCM3',    # 18
        # 'CESM1-BGC'     # 19
        ]
model = models[0]



experiments = [
            'historical',
            # 'rcp85'
            ]
experiment = experiments[0]




institutes = {
    'IPSL-CM5A-MR':'IPSL',
    'GFDL-CM3':'NOAA-GFDL',
    'GISS-E2-H':'NASA-GISS',
    'bcc-csm1-1':'BCC',
    'CNRM-CM5':'CNRM-CERFACS',
    'CCSM4':'NCAR',
    'HadGEM2-AO':'NIMR-KMA',
    'BNU-ESM':'BNU',
    'EC-EARTH':'ICHEC',
    'FGOALS-g2':'LASG-CESS',
    'MPI-ESM-MR':'MPI-M',
    'CMCC-CM':'CMCC',
    'inmcm4':'INM',
    'NorESM1-M':'NCC',
    'CanESM2':'CCCma',
    'MIROC5':'MIROC',
    'HadGEM2-CC':'MOHC',
    'MRI-CGCM3':'MRI',
    'CESM1-BGC':'NSF-DOE-NCAR'
    }
institute = institutes[model]

In [93]:
def concat_files(path_folder, experiment):
    if experiment == 'historical':
        yearEnd_first = 1970
        yearStart_last = 1999

    if experiment == 'rcp85':
        yearEnd_first = 2070
        yearStart_last = 2099

        
    files = [f for f in os.listdir(path_folder) if f.endswith('.nc')]
    if 'Amon' in path_folder:
        files = sorted(files, key=lambda x: x[x.index(".nc")-13:x.index(".nc")-9])
        files = [f for f in files if int(f[f.index(".nc")-13:f.index(".nc")-9]) <= yearStart_last and int(f[f.index(".nc")-6:f.index(".nc")-2]) >= yearEnd_first]
    else:
        files = [f for f in os.listdir(path_folder) if f.endswith('.nc')]
        files = sorted(files, key=lambda x: x[x.index(".nc")-17:x.index(".nc")-13])
        files = [f for f in files if int(f[f.index(".nc")-17:f.index(".nc")-13]) <= yearStart_last and int(f[f.index(".nc")-8:f.index(".nc")-4]) >= yearEnd_first]


    
    path_fileList = []
    for file in files:
        path_fileList = np.append(path_fileList, os.path.join(path_folder, file))

    ds = xr.open_mfdataset(path_fileList, combine='by_coords').sel(time=slice(str(yearEnd_first), str(yearStart_last)),lat=slice(-35,35))

    return ds

In [94]:
path_gen = '/g/data/al33/replicas/CMIP5/combined/'+ institute + '/' + model + '/' + experiment + '/mon/atmos/Amon'
ensemble = 'r1i1p1'
version = os.listdir(os.path.join(path_gen, ensemble))[-1]
variable = 'cl'
path_folder =  os.path.join(path_gen, ensemble, version, variable)
ds = concat_files(path_folder, experiment)

# clouds = ds
# clouds_n = regridder(clouds)

# pressureLevels = ds.a*ds.p0 + ds.b*ds.ps
# pressureLevels_n = regridder(pressureLevels)

# pressureLevels_low = xr.where((pressureLevels_n<=10000e2) & (pressureLevels_n>=600), 1, 0)
# cloud_low = clouds_n*pressureLevels_low
# cloud_low = cloud_low.max(dim='lev')
# cloud_low.attrs['units'] = '%'
# cloud_low.attrs['description'] = 'Maximum cloud fraction (%) from plev: 1000-600 hpa'

# pressureLevels_high = xr.where((pressureLevels_n<=250e2) & (pressureLevels_n>=100), 1, 0)
# cloud_high = clouds_n*pressureLevels_high
# cloud_high = cloud_high.max(dim='lev')
# cloud_high.attrs['units'] = '%'
# cloud_high.attrs['description'] = 'Maximum cloud fraction (%) from plev: 250-100 hpa'

# ds_clouds = xr.Dataset(
#     data_vars = {'clouds': clouds},
#     attrs = {'description': 'metric used to find high clouds and low clouds'}
#     )


In [95]:
def save_file(dataset, folder, fileName):
    os.makedirs(folder, exist_ok=True)
    path = folder + '/' + fileName

    if os.path.exists(path):
        os.remove(path)    
    
    dataset.to_netcdf(path)


In [96]:
folder = '/g/data/k10/cb4968/data/cmip5/ds/GFDL-CM3_orig'

In [97]:
ds

<xarray.Dataset>
Dimensions:     (time: 360, lat: 36, bnds: 2, lon: 144, lev: 48)
Coordinates:
  * lat         (lat) float64 -35.0 -33.0 -31.0 -29.0 ... 29.0 31.0 33.0 35.0
  * lon         (lon) float64 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * lev         (lev) float64 0.9961 0.9874 0.9767 ... 3.801e-05 1.711e-05
  * time        (time) object 1970-01-16 12:00:00 ... 1999-12-16 12:00:00
Dimensions without coordinates: bnds
Data variables: (12/14)
    average_DT  (time) timedelta64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T1  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T2  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    lat_bnds    (time, lat, bnds) float64 dask.array<chunksize=(60, 36, 2), meta=np.ndarray>
    lon_bnds    (time, lon, bnds) float64 dask.array<chunksize=(60, 144, 2), meta=np.ndarray>
    ps          (time, lat, lon) float32 dask.array<chunksize=(60, 36, 144), meta=np.ndarray>
    ...          ...
    lev_bnds    (time, lev, bnds) float64 dask.array<chunksize=(60, 48, 2), meta=np.ndarray>
    a           (time, lev) float64 dask.array<chunksize=(60, 48), meta=np.ndarray>
    a_bnds      (time, lev, bnds) float64 dask.array<chunksize=(60, 48, 2), meta=np.ndarray>
    b           (time, lev) float64 dask.array<chunksize=(60, 48), meta=np.ndarray>
    b_bnds      (time, lev, bnds) float64 dask.array<chunksize=(60, 48, 2), meta=np.ndarray>
    p0          (time) float32 1.013e+05 1.013e+05 ... 1.013e+05 1.013e+05
Attributes: (12/27)
    title:                  NOAA GFDL GFDL-CM3, historical (run 1) experiment...
    institute_id:           NOAA GFDL
    source:                 GFDL-CM3 2010 atmosphere: AM3 (AM3p9,C48L48); sea...
    contact:                gfdl.climate.model.info@noaa.gov
    project_id:             CMIP5
    table_id:               Table Amon (31 Jan 2011)
    ...                     ...
    parent_experiment_id:   piControl
    physics_version:        1
    product:                output1
    institution:            NOAA GFDL(201 Forrestal Rd, Princeton, NJ, 08540)
    history:                File was processed by fremetar (GFDL analog of CM...
    parent_experiment_rip:  r1i1p1

In [98]:
save_wap = True
if save_wap:
    fileName = model + '_clouds_' + experiment + '.nc'
    dataset = ds
    save_file(dataset, folder, fileName)